In [1]:
import xml.etree.ElementTree as ET

In [133]:
from os import path
xml_path = path.relpath("Data/pages_8_47.xml")

In [134]:
with open(xml_path, encoding='utf8') as file:
    tree = ET.parse(file)
    root = tree.getroot()

In [2]:
root.find('./')

<Element '{http://www.tei-c.org/ns/1.0}teiHeader' at 0x000001CEB5704C20>

In [ ]:
root.findall('.//{http://www.tei-c.org/ns/1.0}lb')

In [ ]:
root.findall('.//{http://www.tei-c.org/ns/1.0}hi')

In [23]:
b_text = BS_contents.find_all('ab')
print(b_text)

[<ab facs="#facs_8_tr_1" type="">
<lb facs="#facs_8_tr_1_tl_1" n="N001"/>I. Wegweiser.
               <lb facs="#facs_8_tr_1_tl_2" n="N002"/>Orientirung in Wien.
               <lb facs="#facs_8_tr_1_tl_3" n="N003"/>Rundgang um die Bastei.
               <lb facs="#facs_8_tr_1_tl_4" n="N004"/>Wenn es für jeden Reisenden unerläßlich ist, sich
               <lb facs="#facs_8_tr_1_tl_5" n="N005"/>in einer fremden Stadt zu orlentiren, so ist es we=
               <lb facs="#facs_8_tr_1_tl_6" n="N006"/>nigstens in Wien sehr leicht. Der <hi>Stephansplatz</hi>
<lb facs="#facs_8_tr_1_tl_7" n="N007"/>llegt so ziemlich mitten in der Stadt, und mit Hilfe
               <lb facs="#facs_8_tr_1_tl_8" n="N008"/>des Planes wird man bald die Hauptpulsader der
               <lb facs="#facs_8_tr_1_tl_9" n="N009"/>innern Stadt kennen lernen; diese führt nämlich vom
               <lb facs="#facs_8_tr_1_tl_10" n="N010"/>Burgthor durch die Burg über den <hi>Michaels=</hi>
<lb facs="#facs_8_tr_1_tl_11" n="

In [5]:
words = root.find('.//{http://www.tei-c.org/ns/1.0}lb').tail

In [119]:
print(words)

I. Wegweiser.
               


In [5]:
text = []
for lb in root.findall('.//{http://www.tei-c.org/ns/1.0}lb'):
    #for hi in root.findall('.//{http://www.tei-c.org/ns/1.0}hi'):
        line = lb.tail
        #if line is None:
            #text.append(
        #else: 
        text.append(line)

Problemanalyse: Extrahiert man nur den Tail, kann man die gehighlighteten Keywords nicht einfangen - und das zusammenfügen funktioniert auch nicht, weil sobald hinter lb irgendetwas steht, kein None gelistet wird und dementsprechend das Keyword zumindest nicht automatisiert eingefügt werden kann (zumindest nicht im Text selbst)- 

Darum: erst mal nur die Keywords aus dem xml extrahieren - der rest vom text wird einfach als txt importiert! 

In [74]:
keywords = []
for hi in root.findall('.//{http://www.tei-c.org/ns/1.0}hi'):
    words = hi.text
    keywords.append(words)

In [114]:
import re

text_path = path.relpath("Data/pages_8_47.txt")
with open(text_path, encoding='utf8') as file:
    text = file.read()
    

In [115]:
text = ' '.join(text.split())

-> issues to resolve: 
- Worttrennungen am Ende der Zeile auflösen
- bei allen anderen = durch - ersetzen
- große J durch I ersetzen (z.B. Jnstitut)
- große I mitten im Wort durch kleine i ersetzen (sind doch kleine Ls, also gehts leider nicht :()

In [116]:
text_clean = re.sub(r'(\w)= ([^uo])',r'\g<1>\g<2>', text)

In [122]:
text_clean = re.sub(r'=', r'-', text_clean)

In [117]:
text_clean = re.sub(r'\bJ(?=[^aoeiuä])', r'I', text_clean)
#to avoid substituting big J in Josef or Jägerzeile e.g.
#re.findall('I\w*', text_clean)

In [123]:
print(text_clean[:500])

I. Wegweiser. Orientirung in Wien. Rundgang um die Bastei. Wenn es für jeden Reisenden unerläßlich ist, sich in einer fremden Stadt zu orlentiren, so ist es wenigstens in Wien sehr leicht. Der Stephansplatz llegt so ziemlich mitten in der Stadt, und mit Hilfe des Planes wird man bald die Hauptpulsader der innern Stadt kennen lernen; diese führt nämlich vom Burgthor durch die Burg über den Michaelsplatz, Kohlmarkt, Graben, Stephansplatz, Blschofs- und Rothenthurmstraße zum Rothenthurmthore, und b


Jetzt: NER ausprobieren, eine Überlegung wäre bspw., alle genannten Orte aus dem Text zu extrahieren

In [124]:
import spacy
nlp = spacy.load("de_core_news_sm")

In [125]:
from spacy.language import Language
@Language.component("set_custom_boundaries")
def set_custom_boundaries(doc):
    for token in doc[:-1]:
        if token.text == ";":
            doc[token.i + 1].is_sent_start = False
            return doc

Noch was einfügen, um die Fußnoten rauszubekommen! -> das hängt aber auch mit dem tagging zusammen. 

In [126]:
custom_nlp = spacy.load("de_core_news_sm")
custom_nlp.add_pipe("set_custom_boundaries", before="parser")
intro_to_nlp = custom_nlp(text_clean)
sentences = list(intro_to_nlp.sents)
for sentence in sentences:
    print(sentence)

I. Wegweiser.
Orientirung in Wien.
Rundgang um die Bastei.
Wenn es für jeden Reisenden unerläßlich ist, sich in einer fremden Stadt zu orlentiren, so ist es wenigstens in Wien sehr leicht.
Der Stephansplatz llegt so ziemlich mitten in der Stadt, und mit Hilfe des Planes wird man bald die Hauptpulsader der innern Stadt kennen lernen; diese führt nämlich vom Burgthor durch die Burg über den Michaelsplatz, Kohlmarkt, Graben, Stephansplatz, Blschofs- und Rothenthurmstraße zum Rothenthurmthore, und bildet die Figur einer doppelten Stufe, deren innerer Winkel St. Stephan ist.
Zu einer dieser Straßen, auf elnen dieser Plätze wird man sich bald finden oder erfragen.
— Eine Bestelgung des Stephansthurmes gibt wohl eine vollständige Uebersicht der Stadt und Umgebung, aber zur Orientirung ist sie keineswegs so wesentlich.
Was die Vorstädte anbelangt, so hat man, aus ihnen auf das Glacis hervorkommend, überall den Stephansthurm vor Augen, wird also leicht in eine bestimmte Gegend der Stadt zurück 

Erster einfacher Versuch mit Locations: 

In [131]:
mentioned_locations = []
for ent in intro_to_nlp.ents:    
    if ent.label_ in ['LOC']:
        mentioned_locations.append(ent.text)  

In [132]:
print(mentioned_locations)

['Wien', 'Wien', 'Stephansplatz', 'Stadt', 'Planes', 'Burgthor', 'Kohlmarkt', 'Graben', 'Stephansplatz', 'St. Stephan', 'Stephansthurm', 'Stadt', 'höchstens', 'Rasenparterre', 'Gebäudes', 'Erlach', 'Palais des Fürsten Auersperg', 'Mariahilfer Hauptstraße', 'Schönbrunn', 'Grenadierkaserne', 'Wien', 'Karlskirche', 'Glaswänden', 'Walle', 'Burgplatze', 'Volksgartens', 'Burggartens', 'Remy', 'Flügelsalon', 'Garten', 'Kärnthnerthor-Theater', 'Kärnthnerstraße', 'Wien', 'Wieden', 'Baden', 'Wienerwaldes', 'Schneeberg', 'Unks', 'Karlskirche', 'Schwarzenberg', 'Raaber Eisenbahn', 'Schwarzenberglschen Palais', 'Walle', 'Artillerie-Haupt-Zeugamt', 'Stadt', 'Weihburggasse', 'Mineralwässer-Trinkkur-Anstalt', 'Rennweg', 'Navelin', 'Herzogs von Koburg', 'Walles', 'Stubenthor', 'Stelnbrücke', 'Wien', 'Landstraße', 'St. Marx', 'Ungarn', 'Neustädter Kanal', 'Walle', 'Walle', 'Hauptmauth', 'Walle', 'Ravelin', 'Donaukanal', 'Kahlengebirge', 'Leopoldstadt', 'Taborstraße', 'Tabor', 'Böhmen', 'Donau', 'Franzen

Offensichtlich werden auch Worte wie Planes oder höchstens als Location erkannt - da muss also noch was getan werden. 
Außerdem: Wien und Stadt (alleine) kann raus. 